In [2]:
# Obtiene la curva de distribución acumulada del modelo.

import os

import numpy as np
import pandas as pd

import xarray as xr

In [5]:
vars = [ "Pressure", "Relative_Humidity", "Temperature",
    "Wind_Direction", "Wind_Speed", "DNI", "GHI", "UVHI",
    #"Surface_Albedo", "Precipitable_Water"
    ]
dims = ["XTIME", "south_north", "west_east"]

path_d   = "../temp/NSRDB/CDF_vars_model/"
path_m   = "../temp/TMY/CDF_vars_model/"
path_res = "../temp/TMY/CDF_vars_map/"
path_map = "../temp/TMY/qmap_vars/"

for v in vars:
    print(f"\n{v}")

    # Iteramos para todos los archivos.
    files = os.listdir( path_d + v + "/" )
    files.sort()
    if ".DS_Store" in files: files.remove(".DS_Store")
    for f in files:
        print( f"{f[-5:-3]}", end = "," )
        
        with xr.open_dataset(path_m + v + "/" + f) as ds_m:
            with xr.open_dataset(path_d + v + "/" + f) as ds_d:
                
                ds_m = ds_m.isel( { "south_north": slice(1, 4),
                    "west_east": slice(2, 5) } )
                ds_m["south_north"] = [0, 1, 2]
                ds_m["west_east"  ] = [0, 1, 2]

                df_d = ds_d.to_dataframe()
                df_m = ds_m.to_dataframe()

                df_m["map"] = np.interp( df_m["q_model"],
                    df_d["q_model"], df_d[v] )
                
            df_m["diff_sum"] = df_m[v] - df_m["map"]
            df_m["diff_div"] = df_m[v] / df_m["map"]

            ds_map = df_m.to_xarray()
            ds_m["map"] = ( dims, ds_map["map"].values )
            ds_m["diff_sum"] = ( dims, ds_map["diff_sum"].values )
            ds_m["diff_div"] = ( dims, ds_map["diff_div"].values )

            ds_m.to_netcdf( f"{path_res}{v}/{f}" )
            if not os.path.exists( f"{path_map}{f[:-3]}/" ):
                os.mkdir( f"{path_map}{f[:-3]}/" )
            ds_m[["map"]].rename( {"map": v} ).to_netcdf(
                f"{path_map}{f[:-3]}/{f[:-3]}_{v}.nc" )


Pressure
54,
Relative_Humidity
54,
Temperature
54,
Wind_Direction
54,
Wind_Speed
54,
DNI
54,
GHI
54,
UVHI
54,